In [4]:
import pyarrow.parquet as pq
import numpy as np
import pandas as pd
import pyarrow as pa
import csv

df_2023 = pd.read_parquet("/Users/jesperberglund/Downloads/yellow_tripdata_2023-08.parquet")
df_2022 = pd.read_parquet("/Users/jesperberglund/Downloads/yellow_tripdata_2022-08.parquet")
df_2021 = pd.read_parquet("/Users/jesperberglund/Downloads/yellow_tripdata_2021-08.parquet")
df_2020 = pd.read_parquet("/Users/jesperberglund/Downloads/yellow_tripdata_2020-08.parquet")
df_2019 = pd.read_parquet("/Users/jesperberglund/Downloads/yellow_tripdata_2019-08.parquet")
df_2018 = pd.read_parquet("/Users/jesperberglund/Downloads/yellow_tripdata_2018-08.parquet")
df_2017 = pd.read_parquet("/Users/jesperberglund/Downloads/yellow_tripdata_2017-08.parquet")
df_2016 = pd.read_parquet("/Users/jesperberglund/Downloads/yellow_tripdata_2016-08.parquet")
df_2015 = pd.read_parquet("/Users/jesperberglund/Downloads/yellow_tripdata_2015-08.parquet")
df_2014 = pd.read_parquet("/Users/jesperberglund/Downloads/yellow_tripdata_2014-08.parquet")

df_total = pd.concat([df_2023, df_2022, df_2021, df_2020, df_2019, df_2018, df_2017, df_2016, df_2015, df_2014], axis=1)
# print(df_total.columns)
# print(df_total['tip_amount'])
# h = [x for x in df_total['tip_amount'] if x == 0]
# print(len(h))
pd.DataFrame.to_csv(df_total['tip_amount'][0:1000], 'df_tip.csv')
pd.DataFrame.to_csv(df_total['tip_amount'][0:1000].mean(), 'df_tip_mean.csv')
